# Necesasry Imports:

In [1]:
import numpy as np;
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision
import sklearn.metrics as sk
from collections import OrderedDict
import nbimporter
#from OOD_methods import SMT

ModuleNotFoundError: No module named 'torch'

# Models

In [ ]:
device = torch.device('cuda')
class NewModel(nn.Module):
    def __init__(self, output_layers, *args):
        super().__init__(*args)
        self.output_layers = output_layers
        #print(self.output_layers)
        self.selected_out = OrderedDict()
        #PRETRAINED MODEL
        self.trained_model=models.regnet_y_32gf()
        self.trained_model.fc=torch.nn.Linear(3712,142) 
        self.torchLoad =torch.load(modelName,map_location=torch.device('cuda'))
        self.trained_model.load_state_dict(self.torchLoad['model']) 
 
        self.fhooks = []
        for i,l in enumerate(list(self.trained_model._modules.keys())):
            print("i ==",i,"l ===", l)
            if i in self.output_layers:
                self.layer_name = l;
                print("entered the if clause")
                self.fhooks.append(getattr(self.trained_model,l).register_forward_hook(self.forward_hook(l)))
    
    def forward_hook(self,layer_name):
        def hook(module, input, output):
            self.selected_out[self.layer_name] = output
        return hook

    def forward(self, x):
        out = self.trained_model(x)
        return out, self.selected_out


def returnModel(modelName, mtype):
    if mtype == "SMT":
        trained_model=models.regnet_y_32gf()
        trained_model.fc = torch.nn.Linear(3712,142) 
        torchLoad =torch.load( modelName,map_location=torch.device('cuda'))
        trained_model.load_state_dict(torchLoad['model']) 
        model = trained_model
    if mtype == "MAH":
        model = NewModel(output_layers = [2]).to(device)
    return model;    

# Load datasets

In [ ]:

# datasets locations
inDistValidStr = "/work/baskarg/Mojdeh/iNat_Project-mini-Insecta-2021/Data/in-distribution-valid/Insecta-144c-valid"
outDistValidStr = "/work/baskarg/Mojdeh/iNat_Project-mini-Insecta-2021/Data/out-distribution-valid/noninsecta-2526c-valid/2021_valid"
inDistTrainStr = "/work/baskarg/Mojdeh/iNat_Project-mini-Insecta-2021/Data/in-distribution-train/Insecta-142c-train"


# Loaing the datasets:
data_transform = transforms.Compose([transforms.ToTensor(),transforms.Resize((224,224))])

inDistValid = datasets.ImageFolder( root = inDistValidStr, transform = data_transform)
inDistValid_loader = DataLoader( inDistValid, batch_size = 256, shuffle = False)

inDistTrain = datasets.ImageFolder( root = inDistTrainStr, transform = data_transform)
inDistTrain_loader = DataLoader(inDistTrain, batch_size = 256, shuffle = False)

outDistValid = datasets.ImageFolder(root = outDistValidStr, transform = data_transform)
outDistValid_loader = DataLoader(inDistTrain, batch_size = 256, shuffle = False)


In [ ]:
device = "cuda"
def standalone_get_prelogits( dataLoader):
    prelogits_all = []
    logits_all = []
    labels_all = []
    for inputs, labels in dataLoader:
        inputs, labels = inputs.to(device),labels.to(device)
        labels_np = labels.cpu().detach().numpy()
        if len(labels_all) == 0:
            labels_all = labels_np
        else:
            labels_all = np.concatenate((labels_all,labels_np), axis = 0)   

            
        logits, layersOut = model1(inputs) 
        #print("layersOut ===",layersOut)
        
        prelogits = layersOut['avgpool']
        if len(logits_all) == 0:
            logits_all  =  logits.cpu()
        else:
            logits_all  =  np.concatenate((logits_all,logits.cpu() ), axis = 0)   

        if len(prelogits_all) == 0:
            prelogits_all  =  prelogits.cpu()
        else:
            prelogits_all  =  np.concatenate((prelogits_all,prelogits.cpu()), axis = 0)   
    #return np.concatenate(prelogits_all,axis=0), np.concatenate(logits_all,axis=0), np.concatenate(labels_all,axis=0)
    return prelogits_all, logits_all, labels_all

In [ ]:
with torch.no_grad():
    inDistTrain_embeds, inDistTrain_logits_all, inDistTrain_labels = standalone_get_prelogits(inDistTrain_loader)
    print("finish first")
    del(inDistTrain_logits_all)
    
    torch.save(torch.tensor(inDistTrain_embeds),'inDistTrain_embedsS0M2.pt')
    torch.save(torch.tensor(inDistTrain_labels),'inDistTrain_labelsS0M2.pt')

    inDistValid_embeds, inDistValid_logits_all, inDistValid_labels = standalone_get_prelogits(inDistValid_loader)
    print("finish second")
    accuracy = np.count(arg.max(inDistValid_logits_all) == inDistValid_labels)

    del(inDistValid_logits_all,inDistValid_labels)
    torch.save(torch.tensor(inDistValid_embeds),'inDistValid_embedsS0M2.pt')
    
    outDistValid_embeds, outDistValid_logits_all, outDistValid_labels = standalone_get_prelogits(outDistValid_loader)
    print("finish third")
    del(outDistValid_logits_all,outDistValid_labels)
    torch.save(torch.tensor(outDistValid_embeds),'outDistValid_embedsS0M2.pt')

print(inDistTrain_embeds.shape)
print(inDistTrain_labels.shape)
print("len inDistTrain_loader", len(inDistTrain_loader))
print("len inDistValid_loader", len(inDistValid_loader))
print("len outDistValid_loader", len(outDistValid_loader))



KeyboardInterrupt: 

In [ ]:
print(inDistTrain_labels)

[  0   0   0 ... 141 141 141]


In [ ]:
inDistTrain_embeds = torch.load('inDistTrain_embedsS0M0.pt')
inDistTrain_labels = torch.load('inDistTrain_labelsS0M0.pt')
inDistValid_embeds = torch.load('inDistValid_embedsS0M0.pt')
outDistValid_embeds = torch.load('outDistValid_embedsS0M0.pt')  

# Main

In [ ]:
# I  need to name models in proper way so I could load them structurally. 
modelsPath = "/work/baskarg/Mojdeh/iNat_Project-mini-Insecta-2021/Models/RQ1/"
#modelsNames = ["model_0.pth", "model_1.pth", "model_10.pth", "model_49.pth"]
modelsNames = ["model_1.pth"]

In [ ]:
# we have 4 points in the row and 3 points for the OOD models. 
"""
fig1DF = np.zeros(4,3)
fig2DF = np.zeros(4,3)
fig3DF = np.zeros(4,3)
figs = [fig1DF,fig2DF, fig3DF]
"""

#for i in range(len(modelStrNames)):
# printing each figures:
for name in modelsNames:
#    print(name);
    modelName = modelsPath + name 
    # Load the corresponding 
    print(modelName)
    trained_model = models.regnet_y_32gf()
    trained_model.fc = torch.nn.Linear(3712,142) 
    torchLoad = torch.load( modelName,map_location=torch.device('cuda'))
    trained_model.load_state_dict(torchLoad['model']) 
    model = trained_model
    model0 = returnModel(modelName, "SMT")
    model1 = returnModel(modelName, "MAH")

    #AUROC_S, AUPR_S,FP95_S ,AUROC_E, AUPR_E ,FP95_E = SMT_EBM(model0, inDistTrain_loader, inDistValid_loader, outDistValid_loader)
    #print(AUROC_S, AUPR_S,FP95_S ,AUROC_E, AUPR_E ,FP95_E)
    #print("inDistTrain_labels befpre mah ====", inDistTrain_labels)
    AUROC_M, AUPR_M ,FP95_M = MAH(model1, inDistTrain_embeds,inDistTrain_labels,inDistValid_embeds,outDistValid_embeds)
    #figs[i][j,:] = AUROC1, AUROC2, AUROC3

/work/baskarg/Mojdeh/iNat_Project-mini-Insecta-2021/Models/RQ1/model_1.pth
i == 0 l === stem
i == 1 l === trunk_output
i == 2 l === avgpool
entered the if clause
i == 3 l === fc
inDistTrain_embeds.type <built-in method type of Tensor object at 0x2ae148138098>
inDistTrain_labels before get_scores  ==== tensor([  0,   0,   0,  ..., 141, 141, 141])
indist_train_labels_in inside get_scores ==== tensor([  0,   0,   0,  ..., 141, 141, 141])
pass the normalize_to_unity
indist_train_embeds_in_touse tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0108, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0138],
        [0.0000, 0.0052, 0.0000,  ..., 0.0000, 0.0278, 0.0028],
        ...,
        [0.0000, 0.0000, 0.0022,  ..., 0.0000, 0.0238, 0.0139],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0526],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0046, 0.0000]])
indist_test_embeds_in_touse tensor([[0.0000, 0.0410, 0.0000,  ..., 0.0013, 0.0175, 0.0141],
        [0.000

cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  7.0464e-05, -1.8342e-06,  ..., -1.6440e-07,
         -1.1481e-06,  1.1237e-05],
        [ 0.0000e+00, -1.8342e-06,  1.7978e-05,  ...,  5.3514e-08,
         -3.9618e-06,  6.3090e-07],
        ...,
        [ 0.0000e+00, -1.6440e-07,  5.3514e-08,  ...,  3.5076e-07,
          3.3388e-07, -7.3266e-07],
        [ 0.0000e+00, -1.1481e-06, -3.9618e-06,  ...,  3.3388e-07,
          4.2168e-04, -1.3798e-04],
        [ 0.0000e+00,  1.1237e-05,  6.3090e-07,  ..., -7.3266e-07,
         -1.3798e-04,  5.8128e-04]], dtype=torch.float64)
mean_now ==== tensor([0.0000e+00, 1.7283e-03, 1.8681e-03,  ..., 2.2408e-05, 1.6228e-02,
        2.9176e-02])
cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.1880e-05,  3.9276e-07,  ..., -1.8038e-09,
          5.8255e-06, -8.4707e-06],
        

cov_now === tensor([[ 4.4643e-09, -6.1870e-09,  1.1682e-08,  ..., -9.5336e-10,
         -4.9976e-08,  6.8304e-09],
        [-6.1870e-09,  2.8164e-05, -1.1810e-06,  ..., -9.5817e-08,
          6.4595e-06,  3.9515e-06],
        [ 1.1682e-08, -1.1810e-06,  4.1110e-05,  ..., -3.9192e-07,
         -1.7261e-05,  1.5764e-05],
        ...,
        [-9.5336e-10, -9.5817e-08, -3.9192e-07,  ...,  8.8721e-07,
          5.6816e-07, -3.9523e-06],
        [-4.9976e-08,  6.4595e-06, -1.7261e-05,  ...,  5.6816e-07,
          3.7675e-04, -1.2797e-04],
        [ 6.8304e-09,  3.9515e-06,  1.5764e-05,  ..., -3.9523e-06,
         -1.2797e-04,  8.5865e-04]], dtype=torch.float64)
mean_now ==== tensor([0.0000e+00, 1.7205e-03, 2.2172e-03,  ..., 7.4503e-06, 1.3367e-02,
        1.3825e-02])
cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.1485e-05, -1.1249e-06,  ...,  9.9100e-09,
          8.0176e-06,  4.9097e-06],
        

cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  7.1145e-05,  1.0895e-07,  ...,  5.8665e-08,
          1.1836e-05, -1.4691e-05],
        [ 0.0000e+00,  1.0895e-07,  3.9460e-05,  ...,  3.5707e-08,
         -7.4570e-06,  1.0480e-05],
        ...,
        [ 0.0000e+00,  5.8665e-08,  3.5707e-08,  ...,  1.2028e-07,
          2.9086e-07, -4.3834e-07],
        [ 0.0000e+00,  1.1836e-05, -7.4570e-06,  ...,  2.9086e-07,
          6.3405e-04, -1.4639e-04],
        [ 0.0000e+00, -1.4691e-05,  1.0480e-05,  ..., -4.3834e-07,
         -1.4639e-04,  9.6102e-04]], dtype=torch.float64)
mean_now ==== tensor([0.0000e+00, 3.9915e-03, 6.4324e-03,  ..., 4.1832e-05, 1.6913e-02,
        1.9950e-02])
cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  6.5574e-05, -3.6114e-07,  ..., -2.2875e-08,
          6.1312e-06, -4.1849e-06],
        

cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  3.4656e-05, -4.6958e-06,  ..., -8.9529e-09,
          1.5434e-05, -3.1415e-06],
        [ 0.0000e+00, -4.6958e-06,  1.2494e-04,  ..., -9.9272e-09,
         -2.6751e-05, -4.1856e-06],
        ...,
        [ 0.0000e+00, -8.9529e-09, -9.9272e-09,  ...,  2.6420e-09,
         -2.7039e-08,  3.4648e-08],
        [ 0.0000e+00,  1.5434e-05, -2.6751e-05,  ..., -2.7039e-08,
          3.0004e-04, -5.7336e-05],
        [ 0.0000e+00, -3.1415e-06, -4.1856e-06,  ...,  3.4648e-08,
         -5.7336e-05,  5.4638e-04]], dtype=torch.float64)
mean_now ==== tensor([0.0000e+00, 1.8847e-03, 7.7719e-03,  ..., 2.5213e-06, 1.4129e-02,
        1.8596e-02])
cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.7833e-05, -6.7884e-06,  ..., -4.7578e-09,
          9.8026e-06, -7.3008e-06],
        

...

cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  2.6435e-05, -1.3755e-07,  ..., -2.8747e-08,
          4.6163e-06,  5.5113e-06],
        [ 0.0000e+00, -1.3755e-07,  9.3000e-05,  ..., -2.5955e-08,
         -2.0668e-05,  2.4016e-05],
        ...,
        [ 0.0000e+00, -2.8747e-08, -2.5955e-08,  ...,  4.2320e-08,
          7.2168e-08, -1.7924e-08],
        [ 0.0000e+00,  4.6163e-06, -2.0668e-05,  ...,  7.2168e-08,
          3.2163e-04, -1.4809e-04],
        [ 0.0000e+00,  5.5113e-06,  2.4016e-05,  ..., -1.7924e-08,
         -1.4809e-04,  1.1344e-03]], dtype=torch.float64)
mean_now ==== tensor([0.0000e+00, 1.6448e-03, 1.7044e-03,  ..., 5.0448e-06, 1.7544e-02,
        2.8657e-02])
cov_now === tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.8783e-05, -1.5522e-06,  ..., -3.9588e-09,
          7.9078e-07,  3.8729e-06],
        

in_totrainclasses ==== [tensor([-0.0280, -0.3197, -0.0415,  ..., -0.0080,  0.0393, -0.3146]), tensor([ 0.0215, -0.2569, -0.0483,  ...,  0.0015,  0.0510, -0.2888]), tensor([-0.0408, -0.3300, -0.0663,  ...,  0.0159,  0.1035, -0.3067]), tensor([-0.0452, -0.2671, -0.0543,  ...,  0.0020,  0.1087, -0.2141]), tensor([-0.0105, -0.2790, -0.0717,  ...,  0.0222,  0.0604, -0.2737]), tensor([-0.0363, -0.4394, -0.0274,  ..., -0.0336, -0.0254, -0.4696]), tensor([-0.0129, -0.3880, -0.0585,  ...,  0.0120,  0.0077, -0.3956]), tensor([-0.0090, -0.2888, -0.0337,  ...,  0.0290,  0.0859, -0.2990]), tensor([-0.0470, -0.2989, -0.0359,  ..., -0.0160,  0.0577, -0.2371]), tensor([-0.0058, -0.2869, -0.0397,  ...,  0.0094,  0.0357, -0.2800]), tensor([-0.0437, -0.2592, -0.0589,  ..., -0.0017,  0.0940, -0.1933]), tensor([ 0.0083, -0.2481, -0.0580,  ...,  0.0167,  0.0645, -0.2524]), tensor([-0.0560, -0.2193, -0.0204,  ..., -0.0066,  0.0434, -0.1138]), tensor([-0.0045, -0.2785, -0.0241,  ...,  0.0178,  0.0324, -0.2844

out_scores ==== tensor([ 0.0559, -0.0312,  0.0125,  ..., -0.2243, -0.6155, -0.0023])
in_scores ==== tensor([-0.1080, -0.4649, -0.0966,  ..., -0.0336, -0.0341, -0.4936])
first metrics set:


/home/msaadati/.local/lib/python3.6/site-packages/ipykernel_launcher.py:158: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/msaadati/.local/lib/python3.6/site-packages/ipykernel_launcher.py:159: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 28134 and the array at index 1 has size 112489

In [ ]:
print( AUROC_M, AUPR_M ,FP95_M)

96.25 96.6 0.2000668453951345


# Printing the figures

In [ ]:
# x are accuracy levels, y are aurouc
# change x as needed based on model.
for i in range(len(figs)):
    x1 = [0.25, 0.5,0.75,0.95]
    y1 = figs[i][0,:]
    x2 = [0.25, 0.5,0.75,0.95]
    y2 = figs[i][1,:]
    x3 = [0.25, 0.5,0.75,0.95]
    y3 = figs[i][2,:]
    plt.plot(x1,y1,y1,x2,y2,x3,y3)
    plt.show()

# Utils

In [ ]:
def fpr_and_fdr_at_recall(y_true, y_score, recall_level=95, pos_label=1):
    classes = np.unique(y_true)

    # make y_true a boolean vector
    y_true = (y_true == pos_label)

    # sort scores and corresponding truth values
    desc_score_indices = np.argsort(y_score, kind="mergesort")[::-1]
    y_score = y_score[desc_score_indices]
    y_true = y_true[desc_score_indices]

    # y_score typically has many tied values. Here we extract
    # the indices associated with the distinct values. We also
    # concatenate a value for the end of the curve.
    distinct_value_indices = np.where(np.diff(y_score))[0]
    threshold_idxs = np.r_[distinct_value_indices, y_true.size - 1]

    # accumulate the true positives with decreasing threshold
    tps = np.cumsum(y_true)[threshold_idxs]
    fps = 1 + threshold_idxs - tps      # add one because of zero-based indexing

    thresholds = y_score[threshold_idxs]

    recall = tps / tps[-1]

    last_ind = tps.searchsorted(tps[-1])
    sl = slice(last_ind, None, -1)      # [last_ind::-1]
    recall, fps, tps, thresholds = np.r_[recall[sl], 1], np.r_[fps[sl], 0], np.r_[tps[sl], 0], thresholds[sl]

    cutoff = np.argmin(np.abs(recall - recall_level))

    return fps[cutoff] / (np.sum(np.logical_not(y_true)))

In [ ]:

def return_OOD_metrics_info(safe, risky):
    labels = np.zeros((safe.shape[0] + risky.shape[0]), dtype=np.int32)
    examples = np.squeeze(np.vstack((safe, risky)))
    print("examples.shape ==", examples.shape)
    labels[:safe.shape[0]] += 1
    #print('AUPR (%):', round(100*sk.average_precision_score(labels, examples), 2))
    AUPR = round(100*sk.average_precision_score(labels, examples), 2)
    #print('AUROC (%):', round(100*sk.roc_auc_score(labels, examples), 2))
    AUROC =  round(100*sk.roc_auc_score(labels, examples), 2)
    fpr = fpr_and_fdr_at_recall(labels, examples, 95)
    return  AUROC, AUPR, fpr;


def SMT_EBM(model, inDistTrain_loader, inDistValid_loader, outDistValid_loader):
    # No inference is needed. Prediction  

    # In Distribution data values    
    ################################################################################    
    device = torch.device("cuda")
    model.to(device)
    softmax_indist = []
    energy_indist = []
    inDistValid_y = []
    test_loss = 0
    accuracy = 0
    model.eval()
    CT = 0;
    T = 1;
    print("passed model eval")
    with torch.no_grad():
        for inputs, labels in inDistValid_loader:
            inputs, labels = inputs.to(device),labels.to(device)
            labels_np = labels.cpu().detach().numpy()
            CT = CT + 1;
            if CT > 5:
                break;          
            if len(inDistValid_y) == 0:
                inDistValid_y = labels_np
            else:
                inDistValid_y = np.concatenate((inDistValid_y,labels_np), axis = 0)   
            
            logps = model.forward(inputs)
            sf= torch.nn.Softmax(dim=1)
            softmax = sf(logps)
            softmax = softmax.cpu().detach().numpy()
            if len(softmax_indist) == 0:
                softmax_indist  =  softmax
            else:
                softmax_indist  =  np.concatenate((softmax_indist,softmax), axis = 0)  
            energy = -(T*torch.logsumexp(logps / T, dim=1)).cpu().numpy()   
            energy = np.reshape(energy, (-1, 1))
            if len(energy_indist) == 0:
                energy_indist  =  energy
            else:
                energy_indist  =  np.concatenate((energy_indist,energy), axis = 0)          
    print("finished-IN")    
    # Out distribution data values     
    ################################################################################
    
    device = torch.device("cuda")
    model.to(device)
    softmax_outdist = []
    energy_outdist = []
    outDistValid_y = []
    test_loss = 0
    accuracy = 0
    model.eval()
    CT = 0;
    with torch.no_grad():
        for inputs, labels in outDistValid_loader:
            CT = CT + 1;
            if CT > 5:
                break;
            inputs, labels = inputs.to(device),labels.to(device)
            labels_np = labels.cpu().detach().numpy()
            if len(outDistValid_y) == 0:
                outDistValid_y = labels_np
            else:
                outDistValid_y = np.concatenate((outDistValid_y,labels_np), axis = 0)   
            
            logps = model.forward(inputs)
            sf= torch.nn.Softmax(dim=1)
            softmax = sf(logps)
            softmax = softmax.cpu().detach().numpy()
            if len(softmax_outdist) == 0:
                softmax_outdist  =  softmax
            else:
                softmax_outdist  =  np.concatenate((softmax_outdist,softmax), axis = 0)        
                
            energy = -(T*torch.logsumexp(logps / T, dim=1)).cpu().numpy()   
            energy = np.reshape(energy, (-1, 1))
            if len(energy_outdist) == 0:
                energy_outdist  =  energy
            else:
                energy_outdist  =  np.concatenate((energy_outdist,energy), axis = 0)  
    print("finished-Out")            
    # In Distribution data values    
    ################################################################################    
    def entropy_from_distribution(p, axis):
    #return np.log(10.) + np.sum(p * np.log(np.abs(p) + 1e-11), axis=1, keepdims=True)
        return np.sum(p * np.log(np.abs(p) + 1e-11), axis=1, keepdims=True)
        
    def entropy_stats(softmax):
        s_prob = np.amax(softmax, axis=1, keepdims=True)
        kl_all = entropy_from_distribution(softmax, axis=1)
        mean_all, var_all = np.mean(kl_all), np.var(kl_all)
        return s_prob, kl_all, mean_all, var_all
    
    print("start entropy stat")
    (s_prob_in, kl_in, mean_in, var_in) = entropy_stats(softmax_indist)
    (s_prob_out, kl_out, mean_out, var_out) = entropy_stats(softmax_outdist)
    print("s_prob_in.shape,s_prob_out.shape",s_prob_in.shape,s_prob_out.shape)
    print("energy_indist.shape, energy_outdist.shape",energy_indist.shape, energy_outdist.shape)

    a,b,c = return_OOD_metrics_info(s_prob_in, s_prob_out, False)
    d,e,f = return_OOD_metrics_info(-energy_indist, -energy_outdist, False)
    torch.save(torch.tensor(s_prob_in),'SMT_in_S0M0')
    torch.save(torch.tensor(s_prob_out),'SMT_out_S0M0')

    torch.save(torch.tensor(energy_indist),'EBM_in_S0M0')
    torch.save(torch.tensor(energy_outdist),'EBM_out_S0M0')

    return a,b,c,d,e,f
                

In [ ]:
from sklearn.metrics import roc_auc_score
from sys import getsizeof
from torch import linalg as LA
import pickle;


def get_scores(
    indist_train_embeds_in,
    indist_train_labels_in,
    indist_test_embeds_in,
    outdist_test_embeds_in,
    subtract_mean = True,
    normalize_to_unity = True,
    subtract_train_distance = True,
    indist_classes = 142,
    norm_name = "L2",
    ):
#  
##################################################################################################
  # Normalizing the input
  print("indist_train_labels_in inside get_scores ====", indist_train_labels_in)
  description = ""
  all_train_mean = torch.mean(indist_train_embeds_in,dim=0,keepdims=True)

  indist_train_embeds_in_touse = torch.squeeze(indist_train_embeds_in)
  indist_test_embeds_in_touse = torch.squeeze(indist_test_embeds_in)
  outdist_test_embeds_in_touse = torch.squeeze(outdist_test_embeds_in)


  if subtract_mean:
    indist_train_embeds_in_touse = torch.sub(indist_train_embeds_in_touse, all_train_mean)
    indist_test_embeds_in_touse = torch.sub(indist_test_embeds_in_touse, all_train_mean)
    outdist_test_embeds_in_touse = torch.sub(outdist_test_embeds_in_touse, all_train_mean)
    description = description+" subtract mean,"
    print("pass the subtract_mean")

  if normalize_to_unity:
    indist_train_embeds_in_touse = torch.div(indist_train_embeds_in_touse, LA.norm(indist_train_embeds_in_touse,dim=1,keepdims=True))
    indist_test_embeds_in_touse = torch.div(indist_test_embeds_in_touse , LA.norm(indist_test_embeds_in_touse,dim=1,keepdims=True))
    outdist_test_embeds_in_touse = torch.div(outdist_test_embeds_in_touse, LA.norm(outdist_test_embeds_in_touse,dim=1,keepdims=True))
    description = description + " unit norm,"
    print("pass the normalize_to_unity")  

    
    print("indist_train_embeds_in_touse",indist_train_embeds_in_touse)
    print("indist_test_embeds_in_touse",indist_test_embeds_in_touse)
    print("outdist_test_embeds_in_touse",outdist_test_embeds_in_touse)  
    print("indist_train_labels_in",indist_train_labels_in)
###################################################################################################    
  #Calculate overal mean and covariance of indist train

  maha_intermediate_dict = dict()
  mean = torch.squeeze(torch.mean(indist_train_embeds_in_touse,dim=0))
  meanT = np.cov(torch.t(indist_train_embeds_in_touse - mean).numpy())
  cov = torch.tensor(meanT)
  #cov = cov + 0.01* np.random.rand()
  cov = cov + torch.rand(cov.shape)
  cov_inv = LA.inv(cov) 

  print("train in dist mean", meanT)
  print("train in dist cov_inv", cov_inv)

  #getting per class means and average of per class covariance.
  class_means = []
  class_covs_names = []
  class_covs = []
  i = 0;
  sum = []  
  for c in range(indist_classes):
    mean_now = torch.squeeze(torch.mean(indist_train_embeds_in_touse[indist_train_labels_in == c], dim=0))
    print("mean_now ====", mean_now)
    class_means.append(mean_now)
    cov_now = torch.tensor(np.cov(torch.t(indist_train_embeds_in_touse[indist_train_labels_in == c]-mean_now).numpy()))
    if len(sum) == 0:
         sum = cov_now
    else: 
         sum = sum + cov_now
    print("cov_now ===", cov_now)
  sum = sum/indist_classes
  sum = sum + torch.rand(cov.shape)

  class_cov_invs = LA.inv(sum)
###################################################################################################


  maha_intermediate_dict["class_means"] = class_means
  maha_intermediate_dict["mean"] = mean
  maha_intermediate_dict["cov_inv"] = cov_inv
  maha_intermediate_dict["class_cov_invs"] = class_cov_invs

#################################################################################################### 
# We measure two types of distance. 1- distance to whole distribution. 


  out_totrain = maha_distance(outdist_test_embeds_in_touse,cov_inv,mean,norm_name)
  in_totrain = maha_distance(indist_test_embeds_in_touse,cov_inv,mean,norm_name)
  print("out_totrain ====", out_totrain)
  print("in_totrain ====", in_totrain)
  # Since class cov invs is the same for all classes we omit the index. 
  out_totrainclasses = [maha_distance(outdist_test_embeds_in_touse,class_cov_invs,class_means[c],norm_name) for c in range(indist_classes)]
  in_totrainclasses = [maha_distance(indist_test_embeds_in_touse,class_cov_invs,class_means[c],norm_name) for c in range(indist_classes)]
  print("out_totrainclasses ====", out_totrainclasses)
  print("in_totrainclasses ====", in_totrainclasses)

  # 2- distance to nearest class 
  out_scores = torch.min(torch.stack(out_totrainclasses,dim=0),dim=0)[0]
  in_scores = torch.min(torch.stack(in_totrainclasses,dim=0),dim=0)[0]
  print("out_scores ====", out_scores)
  print("in_scores ====", in_scores)
# Normalization
  if subtract_train_distance:
    out_scores = torch.sub(out_scores , out_totrain)
    in_scores = torch.sub(in_scores, in_totrain)

####################################################################################################

  onehots = torch.Tensor([1]*len(out_scores) + [0]*len(in_scores))
  scores = torch.cat((out_scores,in_scores),dim=0)  
  
  return onehots, scores, description, maha_intermediate_dict, in_scores, out_scores


def maha_distance(xs,cov_inv_in,mean_in,norm_type=None):
    diffs = torch.sub(xs, torch.reshape(mean_in,(1,-1)))
    #second_powers = torch.multiply(torch.matmul(diffs.to(torch.float32),cov_inv_in.to(torch.float32)),diffs.to(torch.float32))
    second_powers = torch.matmul(diffs.to(torch.float32),cov_inv_in.to(torch.float32))*diffs.to(torch.float32)

    if norm_type in [None,"L2"]:
        return torch.sum(second_powers,dim=1)
    elif norm_type in ["L1"]:
        return torch.sum(np.sqrt(np.abs(second_powers)),dim=1)
    elif norm_type in ["Linfty"]:
        return torch.max(second_powers,dim=1)

    
    
def MAH(model,inDistTrain_embeds,
            inDistTrain_labels,
            inDistValid_embeds,
            outDistValid_embeds):
    
    print("inDistTrain_embeds.type", inDistTrain_embeds.type)
#    model, inDistTrain_loader, inDistValid_loader, outDistValid_loader
    print("inDistTrain_labels before get_scores  ====", inDistTrain_labels)
    onehots, scores, description, maha_intermediate_dict, in_scores, out_scores = get_scores(
            inDistTrain_embeds,
            inDistTrain_labels,
            inDistValid_embeds,
            outDistValid_embeds,
            indist_classes = 142,
            subtract_mean = False,
            normalize_to_unity = True,
            subtract_train_distance = True
        )

    onehots = onehots.numpy().reshape((onehots.shape[0], -1))
    scores = scores.numpy().reshape((scores.shape[0], -1))
    torch.save(torch.tensor(in_scores),'MAH_in_S0M1')
    torch.save(torch.tensor(out_scores),'MAH_out_S0M1')
    print("first metrics set:")
    print(return_OOD_metrics_info(in_scores, out_scores))


# Load scores and calculate their OOD measures:

In [ ]:
for strc in range(1):
    for mdl in range(1,2):
        MAH_in_str = "MAH_in_S"+str(strc)+"M"+str(mdl);
        MAH_out_str = "MAH_out_S"+str(strc)+"M"+str(mdl);
        MAH_in = torch.load(MAH_in_str)
        MAH_out = torch.load(MAH_out_str)
        print(MAH_in.shape)
        print(MAH_out.shape)
        MAH_in = MAH_in.reshape((-1,1))
        MAH_out = MAH_out.reshape((-1,1))
        MAH_out = MAH_out[0:MAH_in.shape[0],:]
        print(return_OOD_metrics_info(MAH_in, MAH_out))

torch.Size([28134])
torch.Size([112489])
examples.shape == (56268,)
(48.96, 49.53, 0.9999644558185825)
